In [1]:
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -

pip install --upgrade pip && pip install "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"


In [2]:
!pip install --upgrade pip && pip install "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xtrvmfn5/unsloth_df672108b01e457b91d6081ee56b918c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xtrvmfn5/unsloth_df672108b01e457b91d6081ee56b918c
  Resolved https://github.com/unslothai/unsloth.git to commit 89ba48dd7e5c1d02798d83fa5e347d024ad8c914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [3]:
!pip install "transformers>=4.38.0" "datasets>=2.18.0" "accelerate>=0.28.0" "bitsandbytes>=0.43.0" "trl>=0.8.0"

In [4]:
import torch
from unsloth import FastLanguageModel
import json
from datasets import Dataset
from transformers import TrainingArguments, AutoTokenizer # AutoTokenizer for chat template if needed separately
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
# Configuration
model_name = "google/gemma-2b-it"
your_data_file = "manifesto_fine_tuning_data.json" # Your JSON file
lora_output_name = "lora_model_gemma2b_manifesto_summarizer" # Name for saving adapters
max_seq_length = 2048  # Adjust based on VRAM and data. Gemma 2B context is 8192.
load_in_4bit = True    # Use QLoRA


In [6]:
# Load model and tokenizer using Unsloth
print(f"Loading model: {model_name}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,  # Unsloth will autodetect (e.g., bfloat16 if supported, else float16)
    load_in_4bit=load_in_4bit,
    # token="YOUR_HUGGINGFACE_TOKEN", # Add if you encounter auth issues for Gemma
)
print("Model and tokenizer loaded.")

Loading model: google/gemma-2b-it
==((====))==  Unsloth 2025.7.5: Fast Gemma patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Model and tokenizer loaded.


In [7]:
# Configure LoRA (PEFT)
print("Configuring LoRA adapter...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (8, 16, 32 are common)
    lora_alpha=32,  # Usually 2x r
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth", # Helps save memory
    random_state=3407,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Unsloth often auto-detects these too
    # use_rslora=False, # Advanced
    # loftq_config=None, # Advanced
)
print("LoRA configured.")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Configuring LoRA adapter...


Unsloth 2025.7.5 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


LoRA configured.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

# --- Data Loading and Preparation ---

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# --- Data Loading and Preparation (Revised) ---
print(f"Loading data from: {your_data_file}")
with open(your_data_file, 'r', encoding='utf-8') as f:
    raw_dataset_list = json.load(f)

# Determine which tokenizer to use for templating
# This part is from your previous script's logic
if tokenizer.chat_template is None:
    print("Original tokenizer from FastLanguageModel does not have a chat_template. Loading a fresh one for templating.")
    # Ensure you import AutoTokenizer if you haven't at the top
    # from transformers import AutoTokenizer
    tokenizer_for_template = AutoTokenizer.from_pretrained(model_name)
    # If Gemma requires a specific trust_remote_code for its tokenizer, add it here, though usually not needed for official ones.
    # tokenizer_for_template = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
else:
    print("Using chat_template from the tokenizer loaded by FastLanguageModel.")
    tokenizer_for_template = tokenizer

print("Formatting dataset using chat template...")
formatted_texts = []
for item_index, item in enumerate(raw_dataset_list):
    if "messages" not in item or not isinstance(item["messages"], list):
        print(f"Warning: Item at index {item_index} is missing 'messages' list or is malformed. Skipping.")
        continue

    current_messages = item["messages"]
    messages_for_template = []

    # Check and process the expected structure: system, user, model
    if len(current_messages) == 3 and \
       current_messages[0]["role"] == "system" and \
       current_messages[1]["role"] == "user" and \
       current_messages[2]["role"] == "model":

        system_prompt_content = current_messages[0]["content"]
        user_prompt_content = current_messages[1]["content"]
        model_response_content = current_messages[2]["content"]

        # Combine the system prompt and user prompt into the first "user" turn's content.
        # This is a common way to handle system prompts when the template doesn't support a distinct "system" role.
        combined_user_content = f"{system_prompt_content}\n\n{user_prompt_content}"

        messages_for_template = [
            {"role": "user", "content": combined_user_content},
            {"role": "model", "content": model_response_content}
        ]
    elif len(current_messages) == 2 and \
         current_messages[0]["role"] == "user" and \
         current_messages[1]["role"] == "model":
        # If data is already in user/model format without a system message
        messages_for_template = current_messages
    else:
        # Log unexpected structures and skip
        roles_in_item = [msg.get("role", "unknown_role") for msg in current_messages]
        print(f"Warning: Item at index {item_index} has an unexpected message structure: {roles_in_item}. Skipping.")
        continue

    # Ensure messages_for_template is not empty before applying template
    if not messages_for_template:
        continue

    try:
        # Apply the chat template to the modified messages list
        text = tokenizer_for_template.apply_chat_template(
            messages_for_template,
            tokenize=False,          # We want the string representation
            add_generation_prompt=False  # For training, the prompt should include the model's response
        )
        formatted_texts.append(text + tokenizer_for_template.eos_token) # Append EOS token
    except Exception as e:
        print(f"Error applying chat template at index {item_index} (after potential modification): {e}\nOriginal Item: {item}\nProcessed messages for template: {messages_for_template}")

if not formatted_texts:
    # This will stop the script if no data could be processed, preventing further errors.
    raise ValueError("No data was successfully formatted. Check your JSON structure and error messages. The 'System role not supported' error likely means the chat template for Gemma expects system instructions to be part of the user message or handled differently.")

# Create a Hugging Face Dataset
dataset_dict = {"text": formatted_texts}
dataset = Dataset.from_dict(dataset_dict)

print(f"Dataset prepared. Number of successfully formatted examples: {len(dataset)}")
if len(dataset) > 0:
    print(f"Example formatted text (first successfully processed item):\n{dataset[0]['text']}")

# Optional: Split into train/evaluation sets
# (Using the same logic as before, ensure train_dataset is assigned)
if len(dataset) > 10: # Ensure enough data for a split
    train_eval_split = dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = train_eval_split["train"]
    eval_dataset = train_eval_split["test"] # Make sure this is passed to SFTTrainer if used
    print(f"Train examples: {len(train_dataset)}, Eval examples: {len(eval_dataset)}")
else:
    train_dataset = dataset
    eval_dataset = None
    print("Dataset too small for evaluation split or no data processed, using all available for training (if any).")

Loading data from: manifesto_fine_tuning_data.json
Original tokenizer from FastLanguageModel does not have a chat_template. Loading a fresh one for templating.


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Formatting dataset using chat template...
Dataset prepared. Number of successfully formatted examples: 76
Example formatted text (first successfully processed item):
<bos><start_of_turn>user
You are an AI assistant that provides concise and neutral summaries of election manifesto sections. Focus on key policies and promises.

Please summarize the following section from the election manifesto:

--- Page 0 ---
Bharativajanata Party
Modi
ki
Guarantee
2024
W
Phir EkBaar
2
-
I
A = S
Modi Sarkar
a
a
a
a
a<end_of_turn>
<start_of_turn>model
Line 1: The Bharatiya Janata Party's 2024 election manifesto cover page, titled "Modi ki Guarantee 2024," features the slogan "Phir EkBaar Modi Sarkar."<end_of_turn>
<eos>
Train examples: 68, Eval examples: 8


In [ ]:
# --- Trainer Setup ---
# Ensure you have these imports if you're running this in a new session/cell
# from transformers import TrainingArguments
# from trl import SFTTrainer
# import torch # Should already be imported

print("Setting up Training Arguments...")
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # START WITH 1 for Gemma 2B on typical consumer GPUs. Increase carefully if VRAM allows.
    gradient_accumulation_steps=8,  # Effective batch size = (per_device_train_batch_size * gradient_accumulation_steps). Adjust as needed.
    warmup_ratio=0.1,               # Proportion of training steps for learning rate warmup
    num_train_epochs=1,             # Start with 1-3 epochs. For a first run, 1 epoch is good.
    # max_steps = 50,               # Uncomment for a very quick debug run (trains for only 50 steps)
    learning_rate=2e-4,             # A common learning rate for LoRA
    fp16=not torch.cuda.is_bf16_supported(), # Use fp16 if bf16 is not supported by your GPU
    bf16=torch.cuda.is_bf16_supported(),     # Use bf16 if supported (newer NVIDIA GPUs like Ampere, Hopper)
    logging_steps=5,                # How often to log training loss
    optim="adamw_8bit",             # Optimizer that saves memory, recommended by Unsloth
    weight_decay=0.01,
    lr_scheduler_type="linear",     # Learning rate scheduler type
    seed=3407,                      # For reproducibility
    output_dir="outputs",           # Directory to save checkpoints and final model adapters
    save_strategy="epoch",          # Save a checkpoint at the end of each epoch
    # evaluation_strategy="epoch" if eval_dataset else "no", # Enable if you have an eval_dataset
    # report_to="wandb",            # Optional: if you want to log to Weights & Biases
)

print("Initializing SFTTrainer...")
# Ensure 'model', 'tokenizer', 'train_dataset', and 'max_seq_length' are correctly defined from previous steps
# train_dataset should be the output of your Step 2 (Data Loading and Preparation)
# model and tokenizer should be the LoRA-configured model from Unsloth (Step 1)
# max_seq_length should be consistent with how the model was loaded

if 'train_dataset' not in locals() or train_dataset is None or len(train_dataset) == 0:
    raise ValueError("train_dataset is not defined, is None, or is empty. Please ensure Step 2 (Data Preparation) was successful and train_dataset is populated.")

trainer = SFTTrainer(
    model=model,                     # Your Unsloth LoRA model
    tokenizer=tokenizer,             # Your tokenizer
    train_dataset=train_dataset,     # Your prepared training dataset
    eval_dataset=eval_dataset,       # Your prepared evaluation dataset (if you created one, otherwise set to None or omit)
    dataset_text_field="text",       # The column in your dataset that contains the formatted text strings
    max_seq_length=max_seq_length,   # Max sequence length used during model loading
    dataset_num_proc=2,              # Number of CPU cores for dataset processing (can be adjusted)
    packing=False,                   # Set to True if you have many short sequences and want to pack them for efficiency
    args=training_args,              # The TrainingArguments defined above
)
print("Trainer initialized successfully.")

Setting up Training Arguments...
Initializing SFTTrainer...


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/68 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/8 [00:00<?, ? examples/s]

Trainer initialized successfully.


In [ ]:
# --- Training ---
print("Starting model training...")
try:
    trainer.train()
    print("Training completed successfully.")
except Exception as e:
    print(f"An error occurred during training: {e}")
    # Depending on the error, you might need to adjust batch size, sequence length, or check data.
    raise

Starting model training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 68 | Num Epochs = 1 | Total steps = 8
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 19,611,648/2,000,000,000 (0.98% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: udayatk02 (udayatk02-presidency-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.762100


Training completed successfully.


In [ ]:
# --- Saving Model Adapters ---
# lora_output_name was defined in Step 1, e.g., "lora_model_gemma2b_manifesto_summarizer"
print(f"Saving LoRA adapters to: {lora_output_name}")
model.save_pretrained(lora_output_name)
# tokenizer.save_pretrained(lora_output_name) # Generally not needed to save the tokenizer again for LoRA unless you've made specific changes to it.
print(f"LoRA adapters saved to ./{lora_output_name}")

Saving LoRA adapters to: lora_model_gemma2b_manifesto_summarizer
LoRA adapters saved to ./lora_model_gemma2b_manifesto_summarizer




1.   Prompts Please summarize the following section about economic policy:
2.   Summarize this part regarding healthcare initiatives

1.   Please summarize the key points on education mentioned below
2.   Summarize the defence and security promises in this section


1.   Please summarize the environmental policies outlined here
2.   Extract the main promises regarding job creation from this text



1.   List the specific commitments made towards improving railway infrastructure in this section
2.   Please summarize this section from the Congress manifesto regarding agriculture


1.  Summarize the details provided here about the PM Awas Yojana expansion
2.   What are the key points mentioned about ensuring 'Good Governance' in this section


In [ ]:
# --- Interactive Chat Loop ---
# Ensure necessary libraries are imported
from unsloth import FastLanguageModel
import torch
from transformers import AutoTokenizer, TextStreamer # TextStreamer for nicer output
import os

# Configuration
lora_model_path = "lora_model_gemma2b_manifesto_summarizer" # Your saved adapters path
base_model_name = "google/gemma-2b-it" # The original base model identifier
max_seq_length = 2048  # Should match training
load_in_4bit = True    # Should match training

# --- Load Model and Tokenizer (Done Once) ---
print(f"Loading fine-tuned model from: {lora_model_path}...")
model, _ = FastLanguageModel.from_pretrained(
    model_name=lora_model_path,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=load_in_4bit,
    # token="YOUR_HF_TOKEN", # Add if needed
)
FastLanguageModel.for_inference(model)
print("Model loaded.")

print(f"Loading tokenizer for base model: {base_model_name}...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name) # token="YOUR_HF_TOKEN" if needed)

if tokenizer.chat_template is None:
     # Handle potential missing chat template if necessary (as discussed before)
     # For Gemma, AutoTokenizer should typically load it. If not, investigate/set manually.
     print("WARNING: Chat template not found automatically. Manually setting basic template.")
     # Basic manual template example (verify for Gemma's exact needs if this happens)
     tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '<start_of_turn>user\n' + message['content'] + '<end_of_turn>\n' }}{% elif message['role'] == 'model' %}{{ '<start_of_turn>model\n' + message['content'] + '<end_of_turn>\n' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<start_of_turn>model\n' }}{% endif %}"

print("Tokenizer loaded.")

# Define the System Prompt (Consistent with Training)
system_prompt = "You are an AI assistant that provides concise and neutral summaries of election manifesto sections. Focus on key policies and promises."

# --- Interactive Loop ---
print("\n--- Starting Interactive Chat ---")
print("Type 'quit' or 'exit' to end the chat.")

# Initialize TextStreamer for nice output formatting
# Note: skip_prompt=True is essential here to avoid re-printing the whole input every time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

while True:
    try:
        # 1. Get user input
        user_input_raw = input("You: ")
        if user_input_raw.lower() in ['quit', 'exit']:
            print("Exiting chat.")
            break
        if not user_input_raw.strip(): # Handle empty input
             continue

        # 2. Prepare the prompt using the chat template
        # We'll prepend the system prompt to the user input for this turn.
        # NOTE: This simple loop doesn't maintain conversation history beyond one turn.
        # For multi-turn memory, you'd need to store and pass previous user/model turns
        # in the 'messages_for_inference' list, managing context length carefully.
        combined_user_content = f"{system_prompt}\n\n{user_input_raw}"
        messages_for_inference = [{"role": "user", "content": combined_user_content}]

        inputs = tokenizer.apply_chat_template(
            messages_for_inference,
            tokenize=True,
            add_generation_prompt=True, # Signal model to generate response
            return_tensors="pt"
        ).to(model.device)

        # 3. Generate the response
        print("Model:", end="") # Print header for model's turn, end="" keeps cursor on same line
        _ = model.generate(
            input_ids=inputs,
            streamer=streamer, # Use streamer for output
            max_new_tokens=512, # Adjust max length of response
            use_cache=True,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
        print() # Add a newline after the streamed response is complete

    except KeyboardInterrupt:
        print("\nExiting chat due to interrupt.")
        break
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        # Consider adding more robust error handling if needed
        # break # Optionally break loop on error